### 🕵️‍♂️ Agent Execution Trace: Step-by-Step Reasoning

In this notebook, we executed the agent using the query:

> _"Find the capital of France, then find its current weather condition"_

Because we are using the **LangChain v1.x `create_agent`** factory, the model followed a **ReAct (Reasoning and Acting)** loop. Below is the breakdown of what happened "under the hood" during the execution:

#### 🔄 The Reasoning Loop

1.  **Thought (Reasoning)**: The LLM (Mistral) analyzed the prompt and determined it needed two pieces of external information: the capital of France and the current weather for that specific city.
2.  **Action 1 (Tool Call)**: The agent invoked `duckduckgo_search` with the query `"Capital of France"`.
3.  **Observation 1 (Result)**: The search tool returned a block of text confirming that **Paris** is the capital.
4.  **Action 2 (Tool Call)**: Having identified the city, the agent immediately moved to the next logical step and invoked `get_weather_data` with the argument `city="Paris"`.
5.  **Observation 2 (Result)**: The weather tool returned: `"The current weather in Paris is Overcast. Temperature: 2°C (Feels like -3°C)."`
6.  **Final Response**: The agent synthesized these observations into a natural language response.

#### 📊 Execution Metadata

As seen in the raw result object, the agent successfully managed multiple tool calls in a single run:

- **Model**: `mistral` (via Ollama)
- **Tokens Used**: ~569 Total Tokens
- **Tool Sequence**: `duckduckgo_search` ➔ `get_weather_data`

This demonstrates the power of **Agentic AI**: the model didn't just give a static answer; it dynamically decided which tools to use, in what order, and how to handle the data they returned.


In [ ]:
import requests
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_agent

In [ ]:


# 1. Setup Tools
search_tool = DuckDuckGoSearchRun()

@tool
def get_weather_data(city: str) -> str:
    """
    Fetches current weather data for a city. 
    Input should be a city name (e.g., 'Paris').
    """
    # Note: Using your Weatherstack API key
    api_key = "YOUR_WEATHERSTACK_KEY" 
    url = f"http://api.weatherstack.com/current?access_key={api_key}&query={city}"
    
    try:
        response = requests.get(url)
        data = response.json()
        
        # Weatherstack specific keys
        if "current" in data:
            temp = data['current']['temperature']
            # weather_descriptions is a list
            condition = data['current']['weather_descriptions'][0]
            feels_like = data['current']['feelslike']
            
            return (f"The current weather in {city} is {condition}. "
                    f"Temperature: {temp}°C (Feels like {feels_like}°C).")
        
        elif "error" in data:
            return f"API Error: {data['error']['info']}"
        
        return "Could not parse weather data."
    except Exception as e:
        return f"Request failed: {str(e)}"

In [ ]:


tools = [search_tool, get_weather_data]

# 2. Setup Local LLM (Mistral)
llm = ChatOllama(model="mistral", temperature=0)

# 3. Create Agent
# The system_prompt tells the agent HOW to use the tools.
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(
        "You are a helpful research assistant. "
        "Use the provided tools to answer user queries. "
        "When you need to find information, use the search tool. "
    )
)

# 4. Invoke
# We use 'messages' for v1.x compatibility
result = agent.invoke({
    "messages": [("user", "Find the capital of France, then find its current weather condition")]
})

# 5. Output
print(result["messages"][-1].content)

 It seems there was a mix-up in the responses. The capital of France is indeed Paris, and according to my sources, the current weather in Paris is Overcast with a temperature of 2°C (feels like -3°C).


In [28]:
print(result["messages"])

[HumanMessage(content='Find the capital of France, then find its current weather condition', additional_kwargs={}, response_metadata={}, id='9ef25675-6af6-47a6-8eff-709617fffa63'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2026-02-14T11:22:21.6641057Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2019358100, 'load_duration': 130109400, 'prompt_eval_count': 209, 'prompt_eval_duration': 422851200, 'eval_count': 45, 'eval_duration': 1274664300, 'logprobs': None, 'model_name': 'mistral', 'model_provider': 'ollama'}, id='lc_run--019c5be3-283c-7ec3-a8d7-573adbc66178-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'Capital of France'}, 'id': '568fd6df-ab3f-4999-ae9d-ee857b184ffb', 'type': 'tool_call'}, {'name': 'get_weather_data', 'args': {'city': 'Paris'}, 'id': '17715419-81d4-4ca8-b8cb-c93c1bc8ad7d', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 209, 'output_tokens': 45, 'total